Testing the minimap2 aligner
use conda environment "txomics"
download latest version using conda
You need to open vscode after switching to txomics environment and remap folder in the terminal

In [2]:
! minimap2 --version

2.28-r1209


Download some test FASTQ files
From Roach et. al., 2020
ERR3245465, L1 rep 2
ERR3245468, L3 rep 1
Download from ENA FTP server

In [1]:
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
! wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/008/ERR3245468/ERR3245468.fastq.gz

--2024-11-14 12:10:03--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
           => ‘ERR3245465.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/ERR324/005/ERR3245465 ... done.
==> SIZE ERR3245465.fastq.gz ... 479203040
==> PASV ... done.    ==> RETR ERR3245465.fastq.gz ... done.
Length: 479203040 (457M) (unauthoritative)

ERR3245465.fastq.gz 100%[===================>] 457.00M   686KB/s    in 6m 10s  

2024-11-14 12:15:42 (1.24 MB/s) - Control connection closed.
Retrying.

--2024-11-14 12:29:27--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR324/005/ERR3245465/ERR3245465.fastq.gz
  (try: 2) => ‘ERR3245465.fastq.gz’
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Lo

In [2]:
! mkdir data/
! mv ERR3245465.fastq.gz ERR3245468.fastq.gz data/

Create small fastq files with 1% of reads for testing

In [3]:
! lines=$(zcat data/ERR3245465.fastq.gz | wc -l) && zcat data/ERR3245465.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > data/1pct.ERR3245465.fastq
! lines=$(zcat data/ERR3245468.fastq.gz | wc -l) && zcat data/ERR3245468.fastq.gz | head -n $((((lines / 4) / 100) * 4)) \
    > data/1pct.ERR3245468.fastq


gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [4]:
! cp /home/carl/Code/datacore2024/genome_celegans/1pct* data/

General use command on minimap2 github
Can use gzip fasta and fastq files with minimap2
Some notes from reading the SAM format specification file:
Chimeric alignments are more common on long reads
Linear alignments do not involve strand switching

In [5]:
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq > test_align5.sam
! minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245468.fastq > test_align8.sam

[M::mm_idx_gen::0.020*1.05] collected minimizers
[M::mm_idx_gen::0.026*1.41] sorted minimizers
[M::main::0.026*1.40] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.028*1.38] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.029*1.37] distinct minimizers: 166296 (91.68% are singletons); average occurrences: 1.131; average spacing: 5.330; total length: 1002861
[M::worker_pipeline::0.091*1.94] mapped 5903 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -a data/1pct_elegans.fa data/1pct.ERR3245465.fastq
[M::main] Real time: 0.096 sec; CPU: 0.181 sec; Peak RSS: 0.031 GB
[M::mm_idx_gen::0.019*1.04] collected minimizers
[M::mm_idx_gen::0.025*1.43] sorted minimizers
[M::main::0.025*1.43] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.027*1.40] mid_occ = 31
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.028*1.38] distinct minimizers: 166296 (91.68% are si

Let's look at test_align.sam
I see a lot of bit 4 in the FLAG column, meaning that segment is unmapped.

In [7]:
with open('test_align8.sam', 'r') as fp:
    for line in fp.readlines():
        line = line.rstrip()
        line = line.split('\t')
        if line[1] == '0': 
            print(line)

['ERR3245468.27', '0', 'I', '111520', '60', '154S32M1D33M1I16M2D24M2D4M1I8M1I3M2D25M2D14M1D3M1I15M46D3M2D7M3D28M4D17M1D4M1D5M4D16M1I10M1D12M2I4M1D14M1D4M1D8M1D20M1I12M1I10M1D10M1I12M47D1M1D7M3D10M1D8M2I32M2D29M1D32M3I9M1D16M1I29M1I16M1D8M1I8M1I13M4D14M9S', '*', '0', '0', 'CCTTGAGTTTTGAGGTAAATCAATCGCCTATAATGGCTCCAACAAAGAAAGTGCCACAGGTTCCAGAACTGTGCTCAAGCGCTAGAAAGCAGAGCCGATGCCCTGCACCAAAAGGCTGCTCACTAACACAAGGTGACCGTCGCCGCTAAGAACCAGAAGAAGACCCAAATTTTCAAGCGTGCTGAGAATACGTCCAGGAGTACAGAAACGCCCAAAAGGAAAGGACTCAGACTCAAGCGAGGCTGAGGCCAAGGGAGACTTACGCTTCCAGCTAGAGCAAGGTCGCCTTCGTCATCAGAATTCGGTATCAACCAGCTCATTCCAAAGCCAAGAAAAGGCTCCAGATCCGTTTTCGTCAGATCAACAACGGAGTGTCAAGCTGAACAAGGCATTCACCACCCGTATCATCGAGCCATTACGTAGCTTGGGTTATCCAAACTCATCAGACCATCCACGATACCTTACAAGCGGGATACGCCAAGGTTGACGGAAAACCGTGTCCCAAATCACCGACATACCATCGTGCGAGCAGAGCCTAGAAAGTTCATCATCTGTTTGAGGATCACTGGCCCACAAGATCGCCACCGTCGGACATCATCAAGGAGGCCACCATTTTCTTGTGGCCCTCAAGCTCAACAACCCAACTGGAGGATGGACCAAGAAGAAGACCATCACATCGTTGAAGGAAGGAGACTTCGGAAACAGAGAAGATCAAATTTAACAACCT

Lots of 0's, so many aligned reads

Use look_sam.py. still didin't find any 2s. but why did 272 as a bit flag show up? that's not in the SAM documentation. See this thread:
https://github.com/lh3/minimap2/issues/893
But 272 does not show up in the SAM docs. So how do they know?

Let's use the same parameters used by Roach et. al., 2020 for minimap2
-ax splice -k14 -uf --secondary=no -G 25000 -t 24
Then got SAM files, converted to BAM with SAMtools view -b -F 2048
-uf -k14 are for Nanopore Direct RNA-seq reads
-ax splice long reads
--secondary option for outputing secondary alignments
this means a read aligns well to more than one place, so we don't want those reads
-G is the maximum gap on reference, changes chaining and alignment bandwidth
-t is the number of threads


In [8]:
# this is what the minimap2 documentation recommends 
# for mapping nanopore direct-RNA reads
# let's star with this, change parameters later
# and see how that changes alignments
! minimap2 -ax splice -k14 -uf data/1pct_elegans.fa data/1pct.ERR3245465.fastq > 5aln.sam

[M::mm_idx_gen::0.025*1.03] collected minimizers
[M::mm_idx_gen::0.033*1.43] sorted minimizers
[M::main::0.033*1.43] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::0.036*1.39] mid_occ = 31
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 7
[M::mm_idx_stat::0.038*1.37] distinct minimizers: 291384 (89.75% are singletons); average occurrences: 1.172; average spacing: 2.938; total length: 1002861
[M::worker_pipeline::0.125*2.11] mapped 5903 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -ax splice -k14 -uf data/1pct_elegans.fa data/1pct.ERR3245465.fastq
[M::main] Real time: 0.134 sec; CPU: 0.273 sec; Peak RSS: 0.050 GB


In [ ]:
# this is what was used in Roach et. al.
! minimap2 -ax splice -k14 -uf --secondary=no -G 25000 -t 14 c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz > test_align8.All.sam

[M::mm_idx_gen::1.589*1.34] collected minimizers
[M::mm_idx_gen::1.750*2.43] sorted minimizers
[M::main::1.750*2.43] loaded/built the index for 7 target sequence(s)
[M::mm_mapopt_update::1.928*2.30] mid_occ = 162
[M::mm_idx_stat] kmer size: 14; skip: 5; is_hpc: 0; #seq: 7
[M::mm_idx_stat::2.053*2.22] distinct minimizers: 16602221 (67.22% are singletons); average occurrences: 2.054; average spacing: 2.941; total length: 100286401
[M::worker_pipeline::10.620*8.90] mapped 238012 sequences
[M::main] Version: 2.28-r1209
[M::main] CMD: minimap2 -ax splice -k14 -uf --secondary=no -G 25000 -t 14 c_elegans.PRJNA13758.WS282.genomic.fa.gz data/ERR3245468.fastq.gz
[M::main] Real time: 10.661 sec; CPU: 94.596 sec; Peak RSS: 2.548 GB


still no 2s even with exact parameters. lets move on and learn what are in the generated sam files. how do i organise this data? what do i want to use to compare to the APC dataset?
Some ideas from different papers
How many reads cover the annotated coding sequence? (Roach et. al., 2020)

what are CIGAR strings?
https://jef.works/blog/2017/03/28/CIGAR-strings-for-dummies/
M = match
N = alignment gap
D = deletion
I = insertion
6M = next 6 positions are a match
Alignments use the 0 starting index
splicing events will show gaps

next, what does the phred scale mean?
https://en.wikipedia.org/wiki/Phred_quality_score
probablility of incorrect base call
what is acceptable for long reads?
make a distribution of read quality, and decide a cut off from there

Learning what the SAM flags mean. 0 means the read mapped normally?
https://www.biostars.org/p/7374/
https://broadinstitute.github.io/picard/explain-flags.html

So 2064 is a combination of 0x10 (16) and 0x800 (2048)? 2048 + 16 = 2064
This is what is meant by combination of bitwise flags?
So if everything is 0, it adds up to 0.

use samtools flags for an explanation of what each flag means, like for 2308 which is not in the standard table

what samtools commands sound useful?
http://www.htslib.org/doc/samtools.html

designed to work in a stream with pipes
need to convert to .bam to use view to find things

https://davetang.org/wiki/tiki-index.php?page=SAMTools

In [6]:
# need to sort bam files before creating index
! samtools view -bS test_align5.10.sam > test_align5.10.bam
! samtools sort -o sorted_test_align5.10.bam test_align5.10.bam
! samtools index sorted_test_align5.10.bam sorted_test_align5.10.bai

In [9]:
# run this
! samtools view -bS 5aln.sam > 5aln.bam
! samtools sort -o sorted_5aln.bam 5aln.bam
! samtools index sorted_5aln.bam sorted_5aln.bai

In [11]:
# get all reads mapping to a specific region
! samtools view sorted_5aln.bam IV

# https://github.com/mlafave/sam2pairwise
# makes CIGAR string into nucleotide string

ERR3245465.4322	0	IV	126725	60	8S17M1I4M1D2M1I6M1D8M1D6M2I6M1I5M4D23M1I8M1D12M1D30M1I4M1I7M3D6M510N10M1I11M1I6M1I35M1D9M1I5M1I8M2D9M1I7M1I37M1I3M3I11M1D3M1D1M1D14M1D3M2D1M1D3M1D17M2I2M1D25M1I15M5S	*	0	0	CACGCGAGTTTGAGGATGGTTGCCGCCTTCGCTCACCCGAACATGCTCATTAAATTATGGCATGTCGTTTGGCATCGAATTGATCGCCTCTCGACAGCGTTCTGCCGCCTTTACTTCGGATACGTCCAGCCAAGACACGAGAAAATCACGAACGCTTCGCCAACTACGATCCCATACACCAGAAATGAAGGGAGATTTGTGCCGCGAACAAGGGATACATGCACAATGCCCACAGGGAGCTTCGCCAAGCTACGAGGAGGAAGGGCAAAGACTGTTGCCGATTTGTGGACGCCCACGCCTTGAATTCTTTGTGTGTGTGTTCTGTCATTTTGTTAGTCAATTCAACTTTTGGACTGTTATGATAACTTTAAAATAGCTTGGTTTGTTCCGAATAAAAAATGTTTGTAATTCGAA	#$'#((')*4-,(**%&&/&.(.0.,(%&##$$$$(+.&&'#%'&$$%#%'&)&&'&().((),+&)&+++0+('&%%%/-.-+((),'&&('$(%,('%'$('512*&&(%$#%$(+,-,'&'&'-.'+1&*)(&%+'(.39:--(((()1((%&)&&-0,.)+)-+8.,70)'%%#(+&()5++*/*+-,'(()',''))2)*))1+,*+80+,,,*,-3$&&&&,850)(%')+%(&%)')+-,*(&$'('+(/(**.)-3/')6/,.,./21./3235-/++%,))*.:,'*&-.*$&-).&$)(,''*))(*+*,(#*$&"%#%(*&-6./0/6.*-#$&%(+/.%&%"&'*0))'$'&&#***'.-.())+520.6

In [13]:
# simple stats
! samtools flagstat sorted_test_align5.10.bam

59163 + 0 in total (QC-passed reads + QC-failed reads)
59037 + 0 primary
13 + 0 secondary
113 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
932 + 0 mapped (1.58% : N/A)
806 + 0 primary mapped (1.37% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [13]:
# now need to figure out which reads map to apc genes
# write a seperate python script
! samtools view sorted_5aln.bam > aln_info.txt

Important! The tags at the end of the BAM file like NM:i:84 and cm:i:58 are tags created by minimap2, and are explained here: https://lh3.github.io/minimap2/minimap2.html
DP means dynamic programming?